<a href="https://colab.research.google.com/github/jmand626/PyTorchMLEngine-Transfer-Learning/blob/main/PyTransferLearningPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
torch version: 2.6.0+cu124
torchvision version: 0.21.0+cu124


In [4]:
!git clone https://github.com/jmand626/PyTorchMLEngine-Custom-Dataset-Project.git

fatal: destination path 'PyTorchMLEngine-Custom-Dataset-Project' already exists and is not an empty directory.


In [6]:
%cd PyTorchMLEngine-Custom-Dataset-Project
!ls

/content/PyTorchMLEngine-Custom-Dataset-Project
computer_vision_test_main.py  firsttry_model.py  README.md
create_custom_dataset.py      model_backbone.py  setup_dataholders.py


In [7]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [15]:
import os
import tarfile
import requests
from pathlib import Path

# Define the path to the data folder
data_path = Path("data/")
image_path = data_path / "fgvc_aircraft"

# Create the image directory if it doesn't exist
if not image_path.is_dir():
    print(f"Creating directory at {image_path}...")
    image_path.mkdir(parents=True, exist_ok=True)
else:
    print(f"Directory {image_path} already exists.")

# FGVC Aircraft dataset URL
dataset_url = "https://www.robots.ox.ac.uk/~vgg/data/fgvc-aircraft/archives/fgvc-aircraft-2013b.tar.gz"
dataset_path = data_path / "fgvc-aircraft-2013b.tar.gz"

# Download the dataset if it's not already present
if not dataset_path.is_file():
    print(f"Downloading FGVC Aircraft dataset from {dataset_url}...")
    response = requests.get(dataset_url, stream=True)
    with open(dataset_path, "wb") as file:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)
    print("Download complete.")
else:
    print(f"Dataset already downloaded at {dataset_path}.")

# Extract the dataset
if not any(image_path.iterdir()):
    print(f"Extracting {dataset_path} to {image_path}...")
    with tarfile.open(dataset_path, "r:gz") as tar:
        tar.extractall(path=image_path)
    print("Extraction complete.")
else:
    print(f"Dataset already extracted at {image_path}.")

# Verify extraction
print(f"Contents of {image_path}:")
print(list(image_path.iterdir())[:10])  # Display the first 10 items


Directory data/fgvc_aircraft already exists.
Download complete.
Extracting data/fgvc-aircraft-2013b.tar.gz to data/fgvc_aircraft...
Extraction complete.
Contents of data/fgvc_aircraft:
[PosixPath('data/fgvc_aircraft/fgvc-aircraft-2013b')]


Now hopefully we can continously use the previous setup code whenever we want to use this dataset again.